# Overview of `eolearn.core`

`eolearn.core` is the main subpackage which implements basic building blocks (`EOPatch`, `EOTask` and `EOWorkflow`) and commonly used functionalities.

## EOPatch

EOPatch is common data-object that contains contains multi-temporal remotely sensed data of a single patch (area) of Earth’s surface typically defined by a bounding box in specific coordinate reference system.

There’s no limitation on the amount of data, or the type of data that can be stored. But typically, all of the information is internally stored in form of NumPy arrays as the following features:

- DATA with shape `t x n x m x d`: Time- and position-dependent remote sensing data (e.g. bands) of float type.
- MASK with shape `t x n x m x d`: Time- and position-dependent mask (e.g. ground truth, cloud/shadow mask, super pixel identifier) of integer or boolean type.
- SCALAR with shape `t x d`: Time-dependent and position-independent remote sensing data (e.g. weather data,) of float type.
- LABEL with shape `t x d`: Time-dependent and position-independent label (e.g. ground truth) of integer or boolean type.
- VECTOR: A collection of time-dependent geometry objects stored as a `geopandas.GeoDataFrame` with geometry and `TIMESTAMP` columns.
- DATA_TIMELESS with shape `n x m x d`: Time-independent and position-dependent remote sensing data (e.g. elevation model) of float type.
- MASK_TIMELESS with shape `n x m x d`: Time-independent and position-dependent mask (e.g. ground truth, region of interest mask) of integer or boolean type.
- SCALAR_TIMELESS with shape `d`:  Time-independent and position-independent remote sensing data of float type.
- LABEL_TIMELESS with shape `d`: Time-independent and position-independent label of integer or boolean type.
- VECTOR_TIMELESS: A collection of time-dependent geometry objects stored as a `geopandas.GeoDataFrame` with geometry column.
- META_INFO: A dictionary of additional metadata information (e.g. resolution, time difference).
- BBOX: A bounding box of the patch which is an instance of `sentinelhub.BBox`. It holds information about coordinates and CRS.
- TIMESTAMP: A list of dates of size `t` which are instances of `datetime.datetime` or `datetime.date`.

Note: `t` specifies time component, `n` and `m` are spatical components (number of rows and columns), and `d` is an additional component for data with multiple channels.

Create an empty patch

In [11]:
import xarray as xr
import numpy as np

In [12]:
from eolearn.core import EOPatch

In [13]:
from eolearn.core import FeatureType
from eolearn.core.utilities import array_to_dataframe

In [14]:
from eolearn.core.utilities import eopatch_to_xarrays

In [45]:
patch = EOPatch()

Set a feature to EOPatch. Each feature has to belong to one of the feature types listed above.

In [3]:
import numpy as np
from eolearn.core import FeatureType

In [ ]:
xr.DataArray(np.random.random((2,2)))

In [5]:
patch[FeatureType.DATA]['bands'] = new_bands
# or patch.data['bands'] = new_bands

ValueError: FeatureType.DATA feature has to be a xarray DataArray

Check current content of `EOPatch` with it's string representation.

In [ ]:
patch

Get all non-empty features of EOPatch

In [ ]:
patch.get_features()

Get a feature from EOPatch

In [ ]:
data = patch[FeatureType.DATA]['bands']
# or patch.data['bands']

Save EOPatch to local folder. In case `EOPatch` would already exist in the specified location we are also giving a permission to overwrite its features.

In [ ]:
from eolearn.core import OverwritePermission

patch.save('./example_patch', overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

Load EOPatch from the same folder

In [24]:
patch2 = EOPatch.load('../../example_data/TestEOPatch')

In [25]:
patch2.remove_feature(FeatureType.DATA, "REFERENCE_SCENES")

In [27]:
patch2.data["test"] = np.random.random((68, 101, 100, 1))
patch2

EOPatch(
  data: {
    BANDS-S2-L1C: <xarray.DataArray (dim_0: 68, dim_1: 101, dim_2: 100, dim_3: 13)>
array([[[[0.1007, ..., 0.048 ],
         ...,
         [0.1106, ..., 0.0943]],

        ...,

        [[0.1011, ..., 0.0659],
         ...,
         [0.101 , ..., 0.0645]]],


       ...,


       [[[0.2407, ..., 0.0492],
         ...,
         [0.3224, ..., 0.0586]],

        ...,

        [[0.2275, ..., 0.082 ],
         ...,
         [0.2387, ..., 0.0334]]]], dtype=float32)
Dimensions without coordinates: dim_0, dim_1, dim_2, dim_3
    CLP: <xarray.DataArray (dim_0: 68, dim_1: 101, dim_2: 100, dim_3: 1)>
array([[[[0.004395],
         ...,
         [0.01321 ]],

        ...,

        [[0.002749],
         ...,
         [0.002974]]],


       ...,


       [[[0.165442],
         ...,
         [0.174786]],

        ...,

        [[0.898448],
         ...,
         [0.081008]]]], dtype=float32)
Dimensions without coordinates: dim_0, dim_1, dim_2, dim_3
    NDVI: <xarray.DataArray (dim_

In [28]:
new_eopatch = eopatch_to_xarrays(patch2)
new_eopatch

EOPatch(
  data: {
    BANDS-S2-L1C: <xarray.DataArray 'BANDS-S2-L1C' (time: 68, y: 101, x: 100, depth: 13)>
array([[[[0.1007, ..., 0.048 ],
         ...,
         [0.1106, ..., 0.0943]],

        ...,

        [[0.1011, ..., 0.0659],
         ...,
         [0.101 , ..., 0.0645]]],


       ...,


       [[[0.2407, ..., 0.0492],
         ...,
         [0.3224, ..., 0.0586]],

        ...,

        [[0.2275, ..., 0.082 ],
         ...,
         [0.2387, ..., 0.0334]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2015-07-11T10:00:08 ... 2017-12-22T10:04:15
  * x        (x) float64 4.652e+05 4.652e+05 4.652e+05 ... 4.662e+05 4.662e+05
  * y        (y) float64 5.079e+06 5.079e+06 5.079e+06 ... 5.08e+06 5.08e+06
  * depth    (depth) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    crs:      EPSG:32633
    group:    FeatureType.DATA
    CLP: <xarray.DataArray 'CLP' (time: 68, y: 101, x: 100, depth: 1)>
array([[[[0.004395],
         ...,
         [0.01321 ]],

        ..

In [35]:
for feature in patch2.get_feature_list():
    if not isinstance(feature, tuple):
        continue
    if feature[0] not in (FeatureType.VECTOR, FeatureType.VECTOR_TIMELESS, FeatureType.META_INFO):
        print(patch2[feature[0]][feature[1]])

<xarray.DataArray (dim_0: 5, dim_1: 101, dim_2: 100, dim_3: 13)>
array([[[[0.3623, ..., 0.3031],
         ...,
         [0.3209, ..., 0.2793]],

        ...,

        [[0.3307, ..., 0.2731],
         ...,
         [0.4149, ..., 0.3139]]],


       ...,


       [[[0.1007, ..., 0.048 ],
         ...,
         [0.1106, ..., 0.0943]],

        ...,

        [[0.1011, ..., 0.0659],
         ...,
         [0.101 , ..., 0.0645]]]], dtype=float32)
Dimensions without coordinates: dim_0, dim_1, dim_2, dim_3
<xarray.DataArray (dim_0: 68, dim_1: 101, dim_2: 100, dim_3: 13)>
array([[[[0.1007, ..., 0.048 ],
         ...,
         [0.1106, ..., 0.0943]],

        ...,

        [[0.1011, ..., 0.0659],
         ...,
         [0.101 , ..., 0.0645]]],


       ...,


       [[[0.2407, ..., 0.0492],
         ...,
         [0.3224, ..., 0.0586]],

        ...,

        [[0.2275, ..., 0.082 ],
         ...,
         [0.2387, ..., 0.0334]]]], dtype=float32)
Dimensions without coordinates: dim_0, dim_1, dim_

In [32]:
for feature in patch2.get_feature_list():
    print(type(feature))

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<enum 'FeatureType'>
<enum 'FeatureType'>


Compare EOPatches

In [ ]:
patch == patch2

Remove a feature from EOPatch

In [ ]:
del patch2[FeatureType.DATA]['bands']
# or del patch.data['bands']

In [ ]:
patch2

Make a shallow and deep copy of EOPatch. Shallow copy will copy only a reference to data but not the data itself.

In [ ]:
patch1 = patch.__copy__()
patch2 = patch.__deepcopy__()

patch.data['bands'] += 1

patch == patch1, patch == patch2

Concatenate two EOPatches

In [ ]:
patch2[FeatureType.DATA]['bands2'] = new_bands
patch2

In [ ]:
patch

In [ ]:
patch2

In [ ]:
#patch + patch2
EOPatch.concatenate(patch, patch2)

## EOTask

An EO task is any class the inherits from the abstract `EOTask` class. Each EO task has to implement the execute method; invoking __call__ on a EO task instance invokes the execute method. EO tasks are meant primarily to operate on EO patches (i.e. instances of EOPatch).

Add a feature using the EOTask

In [ ]:
from eolearn.core import AddFeature  # AddFeature is a simple EOTask which adds a feature to a given EOPatch

patch = EOPatch()

feature = (FeatureType.DATA, 'bands')
add_feature = AddFeature(feature)

data = np.zeros((5, 100, 100, 13))
data = xr.
patch = add_feature.execute(patch, data)
# or patch = add_feature(patch, data)

patch

Create a composite task using a multiplication operator (`a * b`) function

In [ ]:
from eolearn.core import CopyTask, RenameFeature

copy_task = CopyTask()
rename_feature = RenameFeature((FeatureType.DATA, 'bands', 'the_bands'))
copy_rename_task = rename_feature * copy_task

new_patch = copy_rename_task(patch)
new_patch

If a task doesn’t exist yet, the user can implement it and easily include it into his/hers workflow. There is very little or almost no overhead in the implementation of a new EOTask as seen from this minimal example

In [ ]:
from eolearn.core import EOTask

class FooTask(EOTask):
    def __init__(self, foo_param):
        self.foo_param = foo_param

    def execute(self, eopatch, *, patch_specific_param=None):
        # do what foo does on input eopatch and return it
        return eopatch

EOTask’s arguments are either static (set when EOTask is initialized; i.e.e foo_param above) or dynamic (set during the execution of the workflow; i.e. patch_specific_param above).

The list of all EOTasks in the `eolearn.core` subpackage is available here https://eo-learn.readthedocs.io/en/latest/eotasks.html#core

## EOWorkflow

A workflow is a directed (acyclic) graph composed of instances of EOTask objects. Each task may take as input the results of other tasks and external arguments. The external arguments are passed anew each time the workflow is executed. The workflow builds the computational graph, performs dependency resolution, and executes the tasks. If the input graph is cyclic, the workflow raises a CyclicDependencyError.

The result of a workflow execution is an immutable mapping from tasks to results. The result contains tasks with zero out-degree (i.e. terminal tasks).

Create a workflow

In [ ]:
from eolearn.core import EOWorkflow, Dependency

workflow = EOWorkflow([
    Dependency(add_feature, inputs=[]),
    Dependency(copy_task, inputs=[add_feature]),
    Dependency(rename_feature, inputs=[copy_task])
])
# Instead of Dependecy class also just a tuple can be used
                                    
result = workflow.execute({
    add_feature: {'eopatch': patch,
                  'data': new_bands}
})
                                    
result

Display the dependency graph

In [ ]:
%matplotlib inline

workflow.dependency_graph('graph.png')

For a linear workflow such as previous one you can also use `LinearWorkflow` class

In [ ]:
from eolearn.core import LinearWorkflow

workflow = LinearWorkflow(add_feature, copy_task, rename_feature)

result = workflow.execute({
    add_feature: {'eopatch': patch,
                  'data': new_bands}
})
                                    
workflow.dependency_graph('graph.png')

## EOExecutor

`EOExecutor` handles execution and monitoring of workflows. It enables executing a workflow multiple times and in parallel. It monitors execution times and handles any error that might occur in the process. At the end it generates a report which contains summary of the workflow and process of execution.

Execute previously defined workflow with different arguments

In [ ]:
from eolearn.core import EOExecutor

execution_args = [  # EOWorkflow will be executed for each of these 3 dictionaries:
    {add_feature: {'eopatch': patch,
                  'data': new_bands}},
    {add_feature: {'eopatch': patch,
                  'data': new_bands - 1}},
    {add_feature: {'eopatch': patch,
                  'data': new_bands * 10}},
]

executor = EOExecutor(workflow, execution_args, save_logs=True, logs_folder='.')

executor.run(workers=3)  # The execution will use at most 3 parallel processes

Make the report

In [ ]:
%matplotlib

executor.make_report()

print('Report was saved to location: {}'.format(executor.get_report_filename()))